https://assets.press.princeton.edu/chapters/s2_10291.pdf

(This is the One-Compartment Model of Repeated Doses, question 3 near end of above pdf)

In [ ]:
import reno

In [ ]:
t = reno.TimeRef()
compartment = reno.Model("one_compartment_model")
with compartment:
    drug_in_system = reno.Stock(doc="mass of medication in the (\"person's blood serum\"?)")
    ingested = reno.Flow(doc="Pulsed inflow of medication when dosage is taken.")
    eliminated = reno.Flow(doc="Rate of change of drug leaving the system.")
    
    absorption_fraction = reno.Variable(.12)
    dosage = reno.Variable(100 * 1000, doc="Dosage is 100 * 1000 micrograms")
    start = reno.Variable(0, doc="Timestep of first dosage. (in hours)")
    interval = reno.Variable(8, doc="Timesteps between each dosage. (in hours)")
    
    volume = reno.Variable(3000, doc="Volume of blood serum, 3000 mL") 
    concentration = reno.Variable(drug_in_system / volume)
    half_life = reno.Variable(22, doc="Half-life of medication. (in hours)")
    elimination_constant = reno.Variable(-reno.log(0.5) / half_life)
    
    eliminated.eq = elimination_constant * drug_in_system
    ingested.eq = absorption_fraction * reno.Piecewise(
        [dosage, 0], 
        [
            ((t - start) >= 0) & reno.eq((t - start) % interval, 0),
            ((t - start) < 0) | reno.ne((t - start) % interval, 0)
        ],
    )

    drug_in_system += ingested
    drug_in_system -= eliminated

In [ ]:
compartment.latex()

In [ ]:
ds = compartment(steps=168)

In [ ]:
reno.plot_trace_refs(compartment, [ds], [compartment.concentration])

In [ ]:
compartment.graph()

In [ ]:
compartment.graph(sparklines=True, sparkall=True)

In [ ]:
is:issue state:open 